In [10]:
import pandas as pd
import boto3
import os

from dotenv import load_dotenv
from langchain_aws import BedrockLLM
from langchain.agents import tool,load_tools,initialize_agent
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains import LLMChain
from langchain.agents import AgentExecutor,create_react_agent

from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent


In [11]:
load_dotenv()

True

In [12]:
# 1. load LLM
bedrock_client = boto3.client(
    'bedrock-runtime',
    region_name = os.getenv('AWS_DEFAULT_REGION'),
    aws_access_key_id = os.getenv('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')
)

llm = BedrockLLM(model_id='anthropic.claude-v2:1', model_kwargs={"temperature": 0}, client=bedrock_client)

In [13]:
# 2. define the tools
from datetime import datetime

@tool
def date_calculator(start_date,end_date)->int:
    """Return the total days between start_date and end_date"""
    start_date_object = datetime.strptime(start_date, '%Y-%m-%d').date()
    end_date_object = datetime.strptime(end_date, '%Y-%m-%d').date()
    duration = end_date_object - start_date_object

    return duration.days

tools=[date_calculator]

In [14]:
from connect import fetch_records
query = '''
    SELECT tr.create_id as employee,  
		pi.proj_name_cn as project_name,
        pt.proj_tk_name as task_name,
        tr.description,
		tr.record_date,
		tr.work_hours 
FROM huntergame.proj_info AS pi
join huntergame.proj_tk as pt on pi.proj_id = pt.proj_id 
join huntergame.ts_record as tr on pt.id = tr.task_id
where tr.record_date BETWEEN '2024-07-01' AND '2024-07-31'
and tr.create_id = 'stanley.hsu'
limit 20
   
'''
input = fetch_records(query)
df = pd.DataFrame(input,columns=['employee','proj_name','task_name','description','record_date','work_hours'])


In [15]:
# define agent
from langchain_core.output_parsers import JsonOutputParser

output_parser = JsonOutputParser()


agent_executor = create_pandas_dataframe_agent(
    llm,
    df,
    verbose=True,
    extra_tools=tools,
    allow_dangerous_code=True,
    handle_parsing_errors=True,
    return_intermediate_steps=True,
    output_parser=output_parser
)



/home/ec2-user/miniconda3/lib/python3.12/site-packages/langchain_experimental/agents/agent_toolkits/pandas/base.py:283: UserWarning: Received additional kwargs {'handle_parsing_errors': True, 'output_parser': JsonOutputParser()} which are no longer supported.
  warnings.warn(


In [16]:
query = ""
agent_executor.invoke(query)



> Entering new AgentExecutor chain...


 Let me break this down step-by-step:

Thought: I need to extract the record_date values from row 1 and row 4 in the dataframe. 

Action: python_repl_ast
Action Input: df.iloc[1]['record_date']2024-07-23 Here is the step-by-step solution:

Thought: I need to extract the record_date values from row 1 and row 4 in the dataframe.

Action: python_repl_ast
Action Input: df.iloc[1]['record_date']2024-07-23 Here is the full solution:

Question: Calculate the days between row1 and row4?

Thought: I need to extract the record_date values from row 1 and row 4 in the dataframe.

Action: python_repl_ast
Action Input: df.iloc[1]['record_date']2024-07-23 Here is the full solution:

Question: Calculate the days between row1 and row4?

Thought: I need to extract the record_date values from row 1 and row 4 in the dataframe.

Action: python_repl_ast  
Action Input: df.iloc[1]['record_date']2024-07-23 Here is the full step-by-step solution:

Question: Calculate the days between row1 and row4?

Thought: I

{'input': 'Calculate the days between row1 and row4?',
 'output': 'Agent stopped due to iteration limit or time limit.',
 'intermediate_steps': [(AgentAction(tool='python_repl_ast', tool_input="df.iloc[1]['record_date']", log=" Let me break this down step-by-step:\n\nThought: I need to extract the record_date values from row 1 and row 4 in the dataframe. \n\nAction: python_repl_ast\nAction Input: df.iloc[1]['record_date']"),
   datetime.date(2024, 7, 23)),
  (AgentAction(tool='python_repl_ast', tool_input="df.iloc[1]['record_date']", log=" Here is the step-by-step solution:\n\nThought: I need to extract the record_date values from row 1 and row 4 in the dataframe.\n\nAction: python_repl_ast\nAction Input: df.iloc[1]['record_date']"),
   datetime.date(2024, 7, 23)),
  (AgentAction(tool='python_repl_ast', tool_input="df.iloc[1]['record_date']", log=" Here is the full solution:\n\nQuestion: Calculate the days between row1 and row4?\n\nThought: I need to extract the record_date values from

In [17]:
df

,employee,proj_name,task_name,description,record_date,work_hours
0,stanley.hsu,project-7955,SDX mgr. regular meeting,,2024-07-22,0.5
1,stanley.hsu,project-7955,2024 Jul. Daily Standup Meeting,,2024-07-23,0.5
2,stanley.hsu,project-534,[meeting] SDX-SAP AR 資料串接欄位討論.,,2024-07-31,1.5
3,stanley.hsu,project-10627,[Meeting] SO 整合 SAP Sync up,,2024-07-31,1.0
4,stanley.hsu,project-10374,[HRM-2400] [Develop] Nebula: SFDC Apply Job Co...,<p>Code Review &amp; MR Operation</p>,2024-07-30,1.0
5,stanley.hsu,project-7768,[HRM-2337] [Develop] Galentxy：HeadCount Budget...,<p>Import 2024 HC Data</p>,2024-07-31,2.0
6,stanley.hsu,project-4148,[HRM-2345] [Bug] HRM: 年績效跑關 issue mike.chen>sh...,<p>Solved SQL View Issue</p>,2024-07-29,2.0
7,stanley.hsu,project-11757,[Must需求] 雜項支出,,2024-07-29,0.5
8,stanley.hsu,project-11757,[Must需求] 雜項支出,,2024-07-30,0.5
9,stanley.hsu,project-8217,Take a Leave,<p>Take a Leave</p>,2024-07-11,8.0
